In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import Proj, transform
import xarray as xr
import geopandas as gpd
from scipy import interpolate

tree_geojson = "../data/data_single_tile/shadow_index/trees_gdf.geojson"
tile_sun_hour = "../data/data_single_tile/berlin_maps_filtered/box_k5_001dom1_33_392_5816_2_be_2021.tiff"
tile_shadow_path = "../data/data_single_tile/shadow_index/paul_linke_2504.csv"

monthly_df = pd.read_csv(tile_shadow_path, index_col=0)
monthly_df.index.name = "tree_id"
monthly_df

In [ ]:
def plot_shadow_indices(df): 
    ax = df.plot(marker='.', figsize=(10,7), title="Shadow index per season", ylabel= "shadow index", xlabel = "days", grid=True)
    ax.set_ylim(0, 1)
    plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
all_shadow_path = "../data/shadow_index/berlin_shadow_box_220323.csv"
all_df = pd.read_csv(all_shadow_path, index_col=0)
all_df.index.name = "tree_id"
all_df

Merge seasonal values with monthly values

In [ ]:
s1 = pd.merge(monthly_df, all_df, how='inner', on=['tree_id'])
s1

In [ ]:
s2 = s1.rename(columns={"winter": "355", "spring": "080", "summer": "172", "autumn": "266"})
seasonal_df = s2[['355', '080', "172", "266"]]
seasonal_df

In [ ]:
seasonal_df = seasonal_df.T
monthly_df = monthly_df.T
monthly_df

Plot monthly shadow indices

In [ ]:
plot_shadow_indices(monthly_df)

Plot seasonal shadow indices

In [ ]:
plot_shadow_indices(seasonal_df)

Prepare a dataframe to interpolate between seasonal values

In [ ]:
inter_seasonal_df = seasonal_df.T
inter_seasonal_df = inter_seasonal_df.reindex(sorted(s2.columns), axis=1)
inter_seasonal_df.rename(columns={"31": "031", "61": "061", "91": "091"},inplace = True)
inter_seasonal_df = inter_seasonal_df.reindex(sorted(inter_seasonal_df.columns), axis=1)
last_column = inter_seasonal_df['355']
new_column = pd.DataFrame({-10: last_column})
inter_seasonal_df= pd.concat([new_column, inter_seasonal_df], axis=1)
inter_seasonal_df

In [ ]:
inter_seasonal_df

In [ ]:
inter_seasonal_dft = inter_seasonal_df.T
inter_seasonal_dft.index = pd.to_numeric(inter_seasonal_dft.index)
lin_interp_df = inter_seasonal_dft.T.interpolate(method ='linear', limit_direction ='backward', axis = 1 )
lin_interp_df = lin_interp_df.drop(columns=[-10, 355, 80, 172, 266])


In [ ]:
lin_interp_df

Plot liner interpolated shadow indices

In [ ]:
plot_shadow_indices(lin_interp_df.T)

Prepare the df for quadratic & cubic interpolation & plot the df

In [ ]:
transformed_seasonal_df = inter_seasonal_df.T
transformed_seasonal_df.index
transformed_seasonal_df.index = pd.to_numeric(transformed_seasonal_df.index)

In [ ]:
cubic_interp_df = transformed_seasonal_df.interpolate(method ='cubic', order=2, limit=None,limit_direction='both' )
cubic_interp_df = cubic_interp_df.T.drop(columns=[-10, 355, 80, 172, 266])
cubic_interp_df 

In [ ]:
quadr_interp_df = transformed_seasonal_df.interpolate(method ='quadratic', limit=None,limit_direction=None)
quadr_interp_df = quadr_interp_df.T.drop(columns=[-10, 355, 80, 172, 266])
quadr_interp_df

Quadratic Intepolated Indices:

In [ ]:
plot_shadow_indices(quadr_interp_df.T)

Cubic interpolated indices: 

In [ ]:
plot_shadow_indices(cubic_interp_df.T)

Calculating the MSE:

In [ ]:
#preparing the dataframes for mse
ground_truth_df = s2.T
lin_interp_dft = lin_interp_df.T

ground_truth_df.index = pd.to_numeric(ground_truth_df.index)
ground_truth_df = ground_truth_df.T.sort_index(axis=1)
ground_truth_df = ground_truth_df.drop(columns=[355, 80, 172, 266])

lin_interp_dft.index = pd.to_numeric(lin_interp_dft.index)
lin_interp_dft = lin_interp_dft.T

quadr_interp_mse = np.mean((ground_truth_df - quadr_interp_df) ** 2)
cubic_interp_mse = np.mean((ground_truth_df - cubic_interp_df) ** 2)
lin_interp_mse = np.mean((ground_truth_df - lin_interp_dft) ** 2)
print("Mean Squared Error (Quadratic Interpolation):", quadr_interp_mse)
print("Mean Squared Error (Cubic Interpolation):", cubic_interp_mse)
print("Mean Squared Error (Cubic Interpolation):", lin_interp_mse)

In [ ]:
quadr_interp_df

In [ ]:
# calculate the overall MSE by averaging across all trees
overall_quadr_interp_mse = np.mean(quadr_interp_mse.values)
overall_cubic_interp_mse = np.mean(cubic_interp_mse.values)
overall_lin_interp_mse = np.mean(lin_interp_mse.values)
print("Overall Mean Squared Error (Quadratic Interpolation):", overall_quadr_interp_mse)
print("Overall Mean Squared Error (Cubic Interpolation):", overall_cubic_interp_mse)
print("Overall Mean Squared Error (Linear Interpolation):", overall_lin_interp_mse)
# compare the overall MSE values 
if overall_quadr_interp_mse < overall_cubic_interp_mse:
    print("Quadratic interpolation has lower overall MSE.")
elif overall_quadr_interp_mse > overall_cubic_interp_mse:
    print("Cubic interpolation has lower overall MSE.")
elif overall_quadr_interp_mse > overall_lin_interp_mse:
    print("Linear interpolation has lower overall MSE.")
else:
    print("Both interpolation methods have the same overall MSE.")

Subplot 3 daraframes: ground truth, linear and cubic interpolation

In [ ]:
# Plot the first tree from each data frame using your preferred style
ax = ground_truth_df.iloc[10, :].plot(marker='.', figsize=(10, 7), title="Shadow Index for First Tree", ylabel="Shadow Index", xlabel="Days", grid = True)
lin_interp_dft.iloc[10,:].plot(ax=ax, marker='.')
cubic_interp_df.iloc[10, :].plot(ax=ax, marker='.')

# Set the y-axis limits
ax.set_ylim(0, 1)

# Set the legend outside the plot area
legend_labels = ['Ground Truth', 'Linear Interpolation', 'Cubic Interpolation']
plt.legend(legend_labels, loc='center left', bbox_to_anchor=(1.0, 0.5))

# Display the plot
plt.show()

In [ ]:
# Specify the tree indices to plot
tree_indices = [0, 1, 2]

# Create a figure with three subplots
fig, axs = plt.subplots(1, 3, figsize=(30, 10))

# Iterate over the tree indices
for i, tree_idx in enumerate(tree_indices):
    # Plot the ground truth values
    axs[i].plot(ground_truth_df.iloc[tree_idx, :], marker='.', label='Ground Truth')
    # Plot the linear interpolation values
    axs[i].plot(lin_interp_dft.T.iloc[:, tree_idx], marker='.', label='Linear Interpolation')
    # Plot the cubic interpolation values
    axs[i].plot(cubic_interp_df.iloc[tree_idx, :], marker='.', label='Cubic Interpolation')

    # Set the y-axis limits
    axs[i].set_ylim(0, 1)

    # Set the title for each subplot
    axs[i].set_title('Tree {}'.format(tree_idx + 1))
    
    # Show the legend only for the first subplot
    if i == 0:
        axs[i].legend(loc='left', bbox_to_anchor=(0, 1))


# Set the common labels and legend
fig.text(0.5, 0.04, 'Days', ha='center')
fig.text(0.04, 0.5, 'Shadow Index', va='center', rotation='vertical')

# Adjust the spacing between subplots
fig.tight_layout()

# Display the plot
plt.show()


In [ ]:
selected_trees_df = gpd.read_file(tree_geojson)
selected_trees_df

In [ ]:
selected_trees_df.rename(columns={"id": "tree_id"}, inplace = True)
selected_trees_df

In [ ]:
#geo_merge = pd.merge(selected_trees_df, seasonal_df, how='inner', on=['tree_id'])

In [ ]:
"""
def mark_trees_in_sunmap(tree_json,sun_hours_map):
    outProj = Proj(init='epsg:25833')
    inProj = Proj(init='epsg:4326')
    sun_hours_map= xr.open_rasterio(sun_hours_map)
    marked_trees_map = sun_hours_map.copy()
    for tree in list(tree_json.items()):
        print(tree)
        lat, lon = tree[1]
        lon, lat = transform(inProj,outProj, lon, lat)
        selected_tree = marked_trees_map.sel(x =[lon],y=[lat], method="nearest")
        marked_trees_map = xr.where((marked_trees_map.x == float(selected_tree.x)) & (marked_trees_map.y == float(selected_tree.y)), 100, marked_trees_map)
    return marked_trees_map
#create json with baumid as key and coordinates as value
data = geo_merge[["tree_id", "geometry"]]
print(data)

selected_trees = {}
for baumid, coordinate in data.itertuples(index=False):
    selected_trees[baumid] = (coordinate.y, coordinate.x)
print(selected_trees)
"""

In [ ]:
#marked_trees_map = mark_trees_in_sunmap(selected_trees, tile_sun_hour)
#marked_trees_map.T.squeeze()[600:800, 800:1100].plot.imshow(cmap="gray")


Cubic Interpolation of all shadow indices for Berlin city trees:

In [ ]:
all_df

In [ ]:
#add montthly columns
monthly_columns = [1, 31, 61, 91, 121, 151, 181, 211, 241, 271, 301, 331]
all_df_monthly = all_df.reindex(columns=[*all_df.columns, *monthly_columns], fill_value=np.nan)
all_df_monthly  = all_df_monthly.rename(columns={"winter": 355, "spring": 80, "summer": 172, "autumn": 266})
all_df_sorted = all_df_monthly.sort_index(axis=1)
#add last column as first column for interpolation
last_column = all_df_sorted[355]
new_column = pd.DataFrame({-10: last_column})
all_df_sorted= pd.concat([new_column, all_df_sorted], axis=1)
all_df_sorted


In [ ]:
all_df_cubic_interp = all_df_sorted.T.interpolate(method ='cubic', order=2, limit=None,limit_direction='both')
all_df_cubic_interp = all_df_cubic_interp.T.drop(columns=[-10, 355, 80, 172, 266])
all_df_cubic_interp  

In [ ]:
all_df_cubic_interp = all_df_cubic_interp.clip(upper=1.0)

In [ ]:
all_df_cubic_interp.to_csv('all_shadow_indices_cubic_interp.csv', index=True)

In [ ]:
all_df = pd.read_csv('all_shadow_indices_cubic_interp.csv', index_col=0)
all_df.index.name = "tree_id"
all_df

In [ ]:
plot_shadow_indices(all_df[2000:2020].T)